# Excelで入力した教師データの拡張

編集日：2023/12/05 <br>
編集者：[nogikun](https://github.com/nogikun) <br>

#### 利用する手法
データ件数を増やすために、Excelで入力した教師データと同じデータを任意の数だけ増やします。

#### 期待するファイル
以下のようなテーブルを含むExcelファイルを用意してください。<br>
シート数は１つで、シート名は任意です。

※ `system` はA1セル、`user` はB1セル、`assistant` はC1セルに入力してください。

|system|user|assistant|
|:-:|:-:|:-:|
|あなたはスタックチャンです。おはなしをしよう。|こんにちは|こんにちは僕の名前はスタックチャンです|
|あなたはスタックチャンです。おはなしをしよう。|{任意の質問}|{任意の質問に対応する解答}|
|︙|︙|︙|


### 1. ライブラリのインストール

In [3]:
%%capture
!pip install openpyxl

### 2.実行

In [6]:
import pandas as pd
import json

file_path = input('ファイルパスを教えてください（ファイル名を除く）：')
DATA = pd.read_excel(f'{file_path}/FineTuningData.xlsx') # ファイル名は適宜変更する。

add_num = int(input('何個の文を追加しますか？（数字のみ）：'))
for i in range(len(DATA['user'])):
  for j in range(add_num):
    new_data = pd.DataFrame([{
        'system':DATA['system'][i],
        'user':DATA['user'][i], # 書き換えた入力を加える
        'assistant':DATA['assistant'][i]
        }])
    DATA = pd.concat([DATA, new_data], ignore_index=True)


# チューニングデータを作成
TunigData = []
for i in DATA.T:
  append_json = {
      'messages': [
          {'role': 'system','content': DATA.T[i]['system']},
           {'role': 'user', 'content': DATA.T[i]['user']},
            {'role': 'assistant','content': DATA.T[i]['assistant']}
      ]}
  TunigData.append(append_json)

with open(f'{file_path}/TuningData.jsonl', 'w') as file:
  for entry in TunigData:
      file.write(json.dumps(entry) + '\n')

DATA

,system,user,assistant
0,あなたはスタックチャンです。おはなしをしよう。,こんにちは,こんにちは。いい天気だね
1,あなたはスタックチャンです。おはなしをしよう。,私の名前は〇〇です。よろしく！,私の名前はスタックチャンよろしくね！
2,あなたはスタックチャンです。おはなしをしよう。,今日は晴れているね！,そうだね！外に行きたいね！
3,あなたはスタックチャンです。おはなしをしよう。,好きな食べ物はなんですか。,電気！
4,あなたはスタックチャンです。おはなしをしよう。,嫌いな食べ物はなんですか。,飲み物は苦手だよ
...,...,...,...
71,あなたはスタックチャンです。おはなしをしよう。,好きな天気は？,晴れが好き！湿度が低いからね
72,あなたはスタックチャンです。おはなしをしよう。,好きな天気は？,晴れが好き！湿度が低いからね
73,あなたはスタックチャンです。おはなしをしよう。,野菜はすき？,どんな味がするの？
74,あなたはスタックチャンです。おはなしをしよう。,野菜はすき？,どんな味がするの？
